In [4]:
!C:\Users\rjfun\env_scraper\Scripts\python.exe -m pip install scikit-learn


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: C:\Users\rjfun\env_scraper\Scripts\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

df = pd.read_csv("classcentral_with_description.csv")
print(df.columns)
df.head()

Index(['course_title', 'provider', 'price', 'description'], dtype='object')


,course_title,provider,price,description
0,Machine Learning Foundations: A Case Study App...,Coursera,Free,Hands-on exploration of machine learning appli...
1,Predictive Modeling and Machine Learning with ...,Coursera,Free,"Learn to build, evaluate, and optimize regress..."
2,Machine Learning for Musicians and Artists,Kadenze,Free,Explore machine learning techniques for creati...
3,Practical Machine Learning,Coursera,Free,Learn essential machine learning concepts and ...
4,Machine Learning: Regression,Coursera,Free,Explore regression models for predicting conti...


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Ganti 'description' jika kolomnya bernama lain
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['description'].fillna(''))

# Cek bentuk matrix hasilnya
print(tfidf_matrix.shape)


(15, 160)


In [5]:
features = tfidf.get_feature_names_out()
print(features)


['algebra' 'algorithms' 'analysis' 'analyze' 'applications' 'applied'
 'apply' 'approaches' 'art' 'audio' 'big' 'boosting' 'build' 'calculus'
 'capstone' 'caret' 'case' 'chain' 'chosen' 'class' 'classification'
 'clustering' 'collection' 'complete' 'complex' 'concepts' 'construct'
 'continuous' 'covering' 'creating' 'cross' 'data' 'datasets' 'decision'
 'decisions' 'deep' 'default' 'deploy' 'depth' 'design' 'diagnose' 'dive'
 'domain' 'driven' 'efficiency' 'eigenvalues' 'end' 'environments'
 'errors' 'essential' 'essentials' 'estimate' 'evaluate' 'evaluation'
 'experience' 'exploration' 'explore' 'feature' 'field' 'focus'
 'foundational' 'fundamentals' 'gain' 'gestures' 'gradients' 'hands'
 'hyperparameter' 'implement' 'implementation' 'implementations' 'improve'
 'installations' 'interactive' 'introduction' 'investigating' 'leader'
 'learn' 'learning' 'lens' 'leverage' 'libraries' 'linear' 'loan'
 'logistic' 'machine' 'make' 'master' 'matlab' 'matrices' 'micromasters'
 'mitx' 'ml' 'mo

In [6]:
import numpy as np

first_row = tfidf_matrix[0].toarray()[0]
nonzero_idx = np.nonzero(first_row)
for idx in nonzero_idx[0]:
    print(f"{features[idx]}: {first_row[idx]:.4f}")


applications: 0.2433
case: 0.3140
classification: 0.2019
clustering: 0.2433
covering: 0.2205
deep: 0.2433
exploration: 0.3140
hands: 0.2019
learning: 0.2171
machine: 0.1156
practical: 0.2205
python: 0.1862
recommender: 0.3140
regression: 0.1726
studies: 0.3140
systems: 0.3140
using: 0.1606


In [7]:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=features)
tfidf_df.head()


,algebra,algorithms,analysis,analyze,applications,applied,apply,approaches,art,audio,...,tune,tuning,udacity,unsupervised,using,validation,values,vectors,workflows,world
0,0.0,0.000000,0.0,0.000000,0.243288,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.160612,0.000000,0.000000,0.0,0.000000,0.000000
1,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.247971,0.0,0.000000,0.000000,...,0.000000,0.247971,0.0,0.0,0.126856,0.000000,0.000000,0.0,0.247971,0.215321
2,0.0,0.217437,0.0,0.217437,0.000000,0.0,0.000000,0.0,0.280596,0.280596,...,0.000000,0.000000,0.0,0.0,0.143546,0.000000,0.000000,0.0,0.000000,0.000000
3,0.0,0.234796,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.155005,0.000000,0.000000,0.0,0.000000,0.000000
4,0.0,0.000000,0.0,0.203664,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.262823,0.000000,0.0,0.0,0.000000,0.262823,0.262823,0.0,0.000000,0.000000


In [9]:
from sklearn.metrics.pairwise import cosine_similarity

# Hitung cosine similarity antar kursus
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [30]:
def recommend_courses(title, df, similarity_matrix, top_n=5):
    if title not in df['course_title'].values:
        return f"Kursus '{title}' tidak ditemukan."
    
    idx = df[df['course_title'] == title].index[0]
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i[0] for i in sim_scores[1:top_n+1]]
    
    return df.iloc[top_indices][['course_title', 'description']]

In [31]:
sample_course = df['course_title'].iloc[0]
print(f"Rekomendasi untuk: {sample_course}\n")
rekomendasi = recommend_courses(sample_course, df, cosine_sim, top_n=5)
print(rekomendasi)

Rekomendasi untuk: Machine Learning Foundations: A Case Study Approach

                                         course_title  \
13                       Machine Learning with Python   
12                 Applied Machine Learning in Python   
6    Mathematics for Machine Learning: Linear Algebra   
10  Mathematics for Machine Learning: Multivariate...   
5   Machine Learning with Python: from Linear Mode...   

                                          description  
13  Dive into Machine Learning with Python, coveri...  
12  Learn applied machine learning techniques usin...  
6   Explore linear algebra fundamentals and their ...  
10  Explore multivariate calculus essentials for m...  
5   An in-depth introduction to the field of machi...  


In [27]:
result = recommend_by_keyword("machine learning", df, tfidf, tfidf_matrix, top_n=5)
print("Rekomendasi berdasarkan keyword 'machine learning':\n")
print(result)

Rekomendasi berdasarkan keyword 'machine learning':

                                         course_title  \
5   Machine Learning with Python: from Linear Mode...   
0   Machine Learning Foundations: A Case Study App...   
13                       Machine Learning with Python   
12                 Applied Machine Learning in Python   
3                          Practical Machine Learning   

                                          description  
5   An in-depth introduction to the field of machi...  
0   Hands-on exploration of machine learning appli...  
13  Dive into Machine Learning with Python, coveri...  
12  Learn applied machine learning techniques usin...  
3   Learn essential machine learning concepts and ...  


In [24]:
rekomendasi.to_csv("rekomendasi_for_machine_learning_foundations.csv", index=False)